In [1]:
import torch
import os
import numpy as np
from dotenv import load_dotenv
from pinecone import Pinecone, PodSpec
from langchain.vectorstores import Pinecone as PineconeStore
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from sentence_transformers import SentenceTransformer
from sentence_transformers import SentenceTransformer

In [2]:
cache_dir="cache_dir"
load_dotenv('./.env')

True

#### Leitura do arquivo e criação do splitter'

In [3]:
with open('docs/nat.txt') as f:
    clt = f.read()

# Criação de um objeto RecursiveCharacterTextSplitter para dividir o texto em pedaços
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=10,
    length_function=len
)

In [4]:
chunks = text_splitter.create_documents([clt])
text_chunks = [doc.page_content for doc in chunks]

In [5]:
# Inicialize o modelo de embedding
model = SentenceTransformer('all-MiniLM-L6-v2')

# Isso retornará uma lista de vetores
embeddings = model.encode(text_chunks, convert_to_tensor=False)  

# Adiciona zeros para estender cada vetor até a dimensão 1536
extended_embeddings = np.array([np.pad(emb, (0, 1536 - len(emb)), 'constant') for emb in embeddings])
print(extended_embeddings[0].shape)

(1536,)


In [6]:
pc = Pinecone(api_key=os.environ.get('PINECONE_API_KEY'))
indexes = pc.list_indexes()
index_name = 'natgpt'

In [7]:
for i in indexes:
    pc.delete_index(i['name'])
    print('Index encontrada e apagada: ' + i['name'])

if index_name not in pc.list_indexes():
    pc.create_index(index_name, dimension=1536, metric='cosine', spec=PodSpec(environment=os.environ.get('PINECONE_ENV')))
    print('Index '+index_name+' criado')

Index encontrada e apagada: natgpt
Index natgpt criado


In [8]:
# Conecta na index criada
pc_index = pc.Index(index_name)

In [9]:
documents_to_insert = []
for i, embedding in enumerate(extended_embeddings):
    doc_id = f"{i}"
    documents_to_insert.append({"id": doc_id, "values": embedding.tolist()})

In [10]:
# Insere os documentos na index do Pinecone
pc_index.upsert(vectors=documents_to_insert)

{'upserted_count': 9}

### Busca por similaridade

In [11]:
# SentenceTransformer para gerar o embedding
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

# Texto para consulta
query_text = "Site do NAT"

# Gerando o vetor de consulta
query_vector = model.encode(query_text, convert_to_tensor=False)

# Adiciona zeros para estender cada vetor até a dimensão 1536
padding_length = 1536 - len(query_vector)
padded_vector = np.pad(query_vector, (0, padding_length), 'constant')
print(padded_vector.shape)

(1536,)


In [12]:
# Convertendo o vetor numpy para lista (caso ainda não esteja em formato de lista)
query_vector_list = padded_vector.tolist()
query_result = pc_index.query(vector=query_vector_list, top_k=3)

print("Query Text Searched: " + query_text)
for index, match in enumerate(query_result.matches):
    print("ID: " + match.id)
    print("Score: " + str(query_result.matches[index]['score']))
    print("Text Value: " + text_chunks[int(match.id)])
    print('-' * 50)

Query Text Searched: Site do NAT
ID: 8
Score: 0.607957661
Text Value: O acesso para registro e acompanhamento de pedidos de apoio ao NAT pode
ser feito por meio do endereço: https://sigep.mpac.mp.br


Site do NAT: https://nat.mpac.mp.br
--------------------------------------------------
ID: 3
Score: 0.450263083
Text Value: Posteriormente, o NAT foi regulado pela nova Lei Orgânica do Ministério
Público do Estado do Acre (Lei Complementar n.º 291, de 29 de dezembro de 2014),
sendo previsto como órgão administrativo auxiliar de apoio técnico especializado de
membros e órgãos do Ministério Público, vinculado à Procuradoria Geral de Justiça.

Desde a sua criação, o NAT vem se mostrando essencial para o
desenvolvimento das atividades finalísticas da instituição.
--------------------------------------------------
ID: 6
Score: 0.442850739
Text Value: O gerenciamento dos pedidos de apoio feitos ao NAT se dá pelo sistema
SIGEP (Sistema de Gerenciamento Eletrônico de Documentos), que consiste em
